In [ ]:
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score 
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, LogisticRegression

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/titanik.csv", delimiter=';')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,Braund; Mr. Owen Harris,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,Cumings; Mrs. John Bradley (Florence Briggs Th...,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,Heikkinen; Miss. Laina,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,Futrelle; Mrs. Jacques Heath (Lily May Peel),female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,Allen; Mr. William Henry,male,35.0,0,0,373450,8.0500,NaN,S


Посмотрим на количество пропусков в столбцах

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


## 1. Вспомогательные преобразования 

* Переведем пол в 0 и 1

In [ ]:
df['Sex'] = df['Sex'].apply(lambda x: 1 if x == 'male' else 0)

* Выкинем столбец `PassengerId`, так как он не несет полезной информации

* Также выкинем столбец `Name` - в рамках этого скринкаста также не будем его использовать

In [ ]:
df.drop(['PassengerId','Name'],axis=1,inplace=True)

* Выкинем столбец `Ticket`, так как это аналог id пассажира - у каждого билета уникальный номер

* Также ***выкинем столбец `Cabin`, так как в нем очень много пропущенных значений (первая стратегия)***

In [ ]:
df.drop(['Ticket','Cabin'],axis=1,inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(5), object(1)
memory usage: 55.8+ KB


Осталось два столбца с пропусками - `Age` и `Embarked`.

***В столбце `Embarked` всего два пропущенных значения - выкинем строки с этими значениями (вторая стратегия).***

In [ ]:
df = df[df['Embarked'].notna()]

Остался столбец `Age` - он представляет наибольший интерес, так как в нем достаточно много пропусков, но и удалять его неразумно. 

Далее посмотрим на различные стратегии заполнения пропусков в этом столбце. Будем заполнять пропуски и смотреть как это отражается на качестве модели.

In [ ]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,S
1,1,1,0,38.0,1,0,71.2833,C
2,1,3,0,26.0,0,0,7.9250,S
3,1,1,0,35.0,1,0,53.1000,S
4,0,3,1,35.0,0,0,8.0500,S


## 2. Кодирование категориальных признаков и масштабирование

Признак `Embarked` может быть полезен, поэтому закодируем его при помощи OneHot encoding.

In [ ]:
df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [ ]:
df = pd.get_dummies(df, columns=['Embarked'], drop_first=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
0,0,3,1,22.0,1,0,7.2500,0,1
1,1,1,0,38.0,1,0,71.2833,0,0
2,1,3,0,26.0,0,0,7.9250,0,1
3,1,1,0,35.0,1,0,53.1000,0,1
4,0,3,1,35.0,0,0,8.0500,0,1


Мы будем обучать линейные модели, поэтому данные важно ***масштабировать***. 

Так как после OHE получились столбцы со значениями 0 и 1, применим для нормализации данных MinMaxScaler, чтобы все значения признаков лежали на отрезке от 0 до 1 (но можно применить и StandardScaler).

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = MinMaxScaler()

df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
0,0.0,1.0,1.0,0.271174,0.125,0.0,0.014151,0.0,1.0
1,1.0,0.0,0.0,0.472229,0.125,0.0,0.139136,0.0,0.0
2,1.0,1.0,0.0,0.321438,0.000,0.0,0.015469,0.0,1.0
3,1.0,0.0,0.0,0.434531,0.125,0.0,0.103644,0.0,1.0
4,0.0,1.0,1.0,0.434531,0.000,0.0,0.015713,0.0,1.0


## 3. Заполнение пропусков в столбце `Age`

* ***Заполним пропуски в столбце `Age` средним значением, но можно еще заполнять медианой (третья стратегия)***

In [ ]:
updated_df = df.copy()
updated_df['Age'] = updated_df['Age'].fillna(updated_df['Age'].mean())

In [ ]:
X = updated_df.drop('Survived', axis=1)
y = updated_df['Survived']

cv = KFold(n_splits=3, random_state=42, shuffle=True)

cross_val_score(LogisticRegression(), X, y, cv=cv).mean()

0.7997732581065914

***Используем машинное обучение для предсказания пропущенных значений (четвертая стратегия)***

In [ ]:
from sklearn.linear_model import LinearRegression

df_numeric = df.copy()

test_df = df_numeric[df_numeric['Age'].isnull() == True]
train_df = df_numeric[df_numeric['Age'].isnull() == False]
y = train_df['Age']

train_df.drop(['Age','Survived'], axis=1, inplace=True)
test_df.drop(['Age','Survived'], axis=1, inplace=True)

In [ ]:
lr = LinearRegression()

lr.fit(train_df, y)

pred = lr.predict(test_df)

In [ ]:
updated_df2 = df.copy()

indices = test_df.index.to_numpy()

updated_df2.loc[indices, 'Age'] = pred

In [ ]:
updated_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889 entries, 0 to 888
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    889 non-null    float64
 1   Pclass      889 non-null    float64
 2   Sex         889 non-null    float64
 3   Age         889 non-null    float64
 4   SibSp       889 non-null    float64
 5   Parch       889 non-null    float64
 6   Fare        889 non-null    float64
 7   Embarked_Q  889 non-null    float64
 8   Embarked_S  889 non-null    float64
dtypes: float64(9)
memory usage: 62.6 KB


In [ ]:
X = updated_df2.drop('Survived', axis=1)
y = updated_df2['Survived']

cross_val_score(LogisticRegression(), X, y, cv=cv).mean()

0.7975210058543393